In [353]:
import spacy
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util
from lexicalrichness import LexicalRichness
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import math
import random

#synonym testing
import numpy as np
nlp = spacy.load('en_core_web_lg')
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize, sent_tokenize
import openai
openai.api_key = 'REPLACE_WITH_YOUR_KEY'
model_engine = "text-davinci-003"

In [300]:
nl = '\n'
class MultipleChoiceQuestion:
    def __init__(self, stem, options, correct_option, qid = None, courseid = None, quality = None):
        self.stem = stem
        self.options = options
        self.correct_option = correct_option
        self.qid = qid
        self.courseid = courseid
        self.quality = quality
        
    def __str__(self):
        return f"Question: {self.stem}\n {nl.join(self.options)}\nCorrect option: {self.correct_option}\nQuality: {self.quality}"

In [378]:
#https://huggingface.co/cointegrated/roberta-large-cola-krishna2020
#Ranking Distractors for Multiple Choice Questions Using Multichannel Semantically Informed CNN-LSTM Networks
cola = pipeline('text-classification', model='cointegrated/roberta-large-cola-krishna2020',truncation=True)

def ambiguous_unclear_information(question):
    output = cola(question.stem)
    score = output[0]['score']
    if score >= 0.7:
        return True
    else:
        print('--- Question stem is unclear')
        return False

In [336]:
#semantic specificity and semantic homogeneity 
#semantic relatedness of answer and other options
#model_gigaword = api.load("glove-wiki-gigaword-100")
#model_gigaword.most_similar(positive=['dirty','grimy'],topn=10)
#v_apple = model_gigaword['Paris']
#v_mango = model_gigaword['London']
#cosine_similarity([v_apple],[v_mango])

#https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
#Distractor Quality Evaluation in Multiple Choice Questions
#Ranking Distractors for Multiple Choice Questions Using Multichannel Semantically Informed CNN-LSTM Networks
model = SentenceTransformer('all-MiniLM-L6-v2')


#Uses NER, so if the score is too low, if they're matching entities (i.e. people) then we can ignore this case and say True
#For example, these two are similar, they're just both entities not recognized. 
#--- distractor not similar enough
#Millikan 		 Rutherford 		 Score: 0.1707
def implausible_distractors(question):
    correct = question.correct_option
    options = question.options.copy()
    options.remove(correct)

    # Two lists of sentences
    sentences1 = [correct, correct, correct]
    sentences2 = options

    #Compute embedding for both lists
    embeddings1 = model.encode(sentences1, convert_to_tensor=True)
    embeddings2 = model.encode(sentences2, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)

    #Output the pairs with their score
    for i in range(len(sentences1)):
        if cosine_scores[i][i] < 0.15: #Was .2
            
            #NER check here...
            opt_entity = nlp(sentences2[i])
            lemma_nouns_opt = get_lemma_nouns(sentences2[i])
            print('opt_entity: ', opt_entity)
            print('lemma_nouns_opt: ', lemma_nouns_opt)
            
            ans_entity = nlp(sentences1[i])
            lemma_nouns_ans = get_lemma_nouns(sentences1[i])
            print('ans_entity: ', ans_entity)
            print('lemma_nouns_ans: ', lemma_nouns_ans)
            

            #If the noun(s) in the answer choice can be tagged with an entity
            if ans_entity.ents:
                answer_entity = ans_entity.ents[0].label_
            else:
                answer_entity = None

            if opt_entity.ents:
                opt_entity = opt_entity.ents[0].label_
            else:
                opt_entity = None

            if answer_entity and opt_entity and answer_entity in opt_entity:
                print('*** low score, but they are the same entity', answer_entity, ' & ', opt_entity)
                return True
            
            if len(lemma_nouns_ans) == 0 and len(lemma_nouns_opt) == 0:
                #Couldn't find the noun nor the entity? Unable to parse effectively to make a judgement.
                print('*** no noun or entity, cant judge')
                return True
            
            #If the option in this case is none/all of the above, it won't be similar, so ignore this criteria
            if not all_of_the_above(question) or not none_of_the_above(question):
                return True
            
            print('--- distractor not similar enough')
            print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))
            return False
        
    return True

In [303]:
def none_of_the_above(question):
    print(question)
    for opt in question.options:

        cleaned_opt = opt.strip().lower()
        if 'none of the above' in cleaned_opt or ('none' in cleaned_opt and 'above' in cleaned_opt) or cleaned_opt == 'neither' or cleaned_opt == "none" or 'none' in question.options[3]:
            print('--- none of the above')
            return False
    
    return True

In [304]:
#If the correct answer is noticably longer (20% or more) than the second longest answer, flag it.
def longest_answer_correct(question):
    correct = question.correct_option
    options = question.options.copy()
    options.remove(correct)
    
    longest_option = 0
    for opt in options:
        if len(opt) >= longest_option:
            longest_option = len(opt)
        
    #If the longest option is only by 20% or it's a single word/number, then this passes
    if longest_option >= len(correct) * 0.8 or len(correct.split()) == 1:
        return True
    
    print('--- longest option is correct')
    return False

In [305]:
def gratuitous_information_in_stem(question):  
    #How effective are lexical richness measures for differentiations of vocabulary proficiency? A comprehensive examination with clustering analysis
    #https://github.com/LSYS/LexicalRichness
    #If the lexical richness is too wild...it's gratutious 
    stem = LexicalRichness(question.stem)
    
    if stem.cttr > 4.5:
        print("--- CTTR above 4.5, text is too complex and extraneous: ", stem.cttr)
        return False
    
    return True

  

In [306]:
s = "This is a test. Here is some extraneous information. Which of these is not true?"
for sent in s.split('.'):
    sent = sent.lower()
    print('sent: ' , sent)
    if 'which' in sent and 'not' in sent:
        print('negative word')

sent:  this is a test
sent:   here is some extraneous information
sent:   which of these is not true?
negative word


In [307]:
#Question should not be a series of true/false statements, so we can look for "which" and "true" or "false" in the stem
def true_or_false(question):
    correct = question.correct_option
    options = question.options.copy()
    options.remove(correct)
    
    #Check for neatively worded stem too.
    for sent in question.stem.split('.'):
        sent = sent.lower()
        if 'which' in sent and ('false' in sent or 'true' in sent):
            return False    
    
    for opt in options:
        if opt.strip().lower() == 'true' or opt.strip().lower() == 'false' or opt.strip().lower() == 'yes'or opt.strip().lower() == 'no':
            print('--- There is a true/false or yes/no answer choice')
            return False

    return True

In [308]:
def avoid_convergence_cues(question):
    #Check for synonyms, because they'll know it's the word they've most recently come across in the text
    #The correct option is likely to be used more (when in pairs, etc.) --> k-type (super similar by description)
    #https://www.rcpch.ac.uk/sites/default/files/rcpch/HTWQ/convergence.html
    
    #So here we check for synonyms used in the words, in case they get lazy with distractors
    options = question.options.copy()
    options.remove(question.correct_option)
    if len(options) < 3:
        return True
    
    lemma_nouns_answ = get_lemma_nouns(question.correct_option)
    lemma_nouns_options = [get_lemma_nouns(options[0]), get_lemma_nouns(options[1]), get_lemma_nouns(options[2])]
    
    #so we want this code, but like, for synonyms
    synonyms = []
    for noun in lemma_nouns_answ:
        for syn in wn.synsets(noun):
            for l in syn.lemmas():
                synonyms.append(l.name().lower().replace('_', ' '))
    
    for opt in lemma_nouns_options:
        repeating_nouns_synonyms = list(set(synonyms).intersection(opt))
        print("repeating_nouns_synonyms: ", repeating_nouns_synonyms)
        if len(repeating_nouns_synonyms) > 0:           
            
            #if the repeat is not in every answer choice, flag it.
            for rns in repeating_nouns_synonyms:           
                
                flag = True
                for value in lemma_nouns_options:
                    if rns not in value:
                        print('--- we have a synonym of the answer being used in other answer choices, but not all of them: ', rns)
                        return False
    
    return True

In [309]:
#An example of a logical cue is asking students to select the most appropriate pharmaceutical intervention for a problem and
#only having one or two options which

def avoid_logical_cues(question):
    #NER? If the question asks for a <certain type of noun, like a person> then the options should all be <people> too.
    #so do NER on the answer and see if we get those in the other options
    
    options = question.options.copy()
    options.remove(question.correct_option)
    if len(options) < 3:
        return True
    
    lemma_nouns_answ = get_lemma_nouns(question.correct_option)
    lemma_nouns_options = [get_lemma_nouns(options[0]), get_lemma_nouns(options[1]), get_lemma_nouns(options[2])]
    
    entities_in_options = []
    for opt in lemma_nouns_options:
        for val in opt:
            doc = nlp(val)
            if doc.ents:
                entities_in_options.append(doc.ents[0].label_)
    
    entities_in_answer = []
  
    for val in lemma_nouns_answ:
        doc = nlp(val)
        
        #If the noun(s) in the answer choice can be tagged with an entity
        if doc.ents:
            answer_entity = doc.ents[0].label_
            if answer_entity not in entities_in_options:
                print('--- The answer entity is not found in any other options: ', answer_entity)
                return False 
    
    return True

In [310]:
def all_of_the_above(question):
    for opt in question.options:
        if 'all of the above' in opt or ('all' in opt and 'above' in opt) or ('all if the' in opt):
            print('--- all of the above')
            return False

    return True

In [311]:
#Might change to be more than a single underscore? For example, __
def fill_in_the_blank(question):
    if "_" in question.stem:
        print('--- fill in the blank')
        return False
    
    return True

In [312]:
#This one might need POS tagging too, because an absolute term isn't a deal breaker in some cases
# i.e. "Why does Hydrogen never bond with...?" 
def absolute_terms(question):
    absolutes = ["always", "never", "every", "none", "only"]
    for opt in question.options:
        #Count all, but not in the case of "all of the above"
        if any(word in opt for word in absolutes) or ("all" in opt and all_of_the_above(question)):
            print('--- absolute word in question stem')
            return False
    
    return True

In [313]:
#Find the nouns in question.correct_option and question.stem --> stem them --> compare cosine similiary (usin sentence transformer)
#Also check for the synonyms, compare them. However, if the word(s) are used in the other options, then it's fine.
#PoS tagging -> lemma -> Synonyms -> Cosine similarity 
#Technically I could check synonyms between stem and other answer choices, but I wont.

lemmatizer = WordNetLemmatizer()
#Nouns: NN noun, singular ‘- desk’, NNS noun plural – ‘desks’, NNP proper noun, singular – ‘Harrison’, NNPS proper noun, plural – ‘Americans’ 
nouns = ['NN', 'NNS', 'NNP', 'NNPS']

def word_repeats_in_stem_and_correct_answer(question):   
    lemma_nouns_stem = get_lemma_nouns(question.stem)        
    lemma_nouns_answ = get_lemma_nouns(question.correct_option)
    
    repeating_nouns = list(set(lemma_nouns_stem).intersection(lemma_nouns_answ))
    
    #check for synonms in question stem w/ answer choice
    synonyms = []
    for noun in lemma_nouns_stem:
        for syn in wn.synsets(noun):
            for l in syn.lemmas():
                synonyms.append(l.name().lower().replace('_', ' '))
                
    repeating_nouns_synonyms = list(set(synonyms).intersection(lemma_nouns_answ))
    
    #If we get a repeat, then it should also repeat in the other answer choices, not just the correct!
    if len(repeating_nouns) > 0 or len(repeating_nouns_synonyms) > 0:
        options = question.options.copy()
        options.remove(question.correct_option)
        options_that_share_noun = 0
        
        for opt in options:
            lemma_option = get_lemma_nouns(opt)
            repeating_nouns_ans_opt = list(set(lemma_option).intersection(lemma_nouns_answ))
            if len(repeating_nouns_ans_opt) > 0:
                options_that_share_noun += 1
        
        #If the word is shared between all options, then it is fine
        if options_that_share_noun == 3 or all_of_the_above(question) or none_of_the_above(question):
            return True
        else:
            print('--- The noun is only shared in certain words')
            return False
        
    #Check for word (adjective, noun, verb, adverb) that repeats just in stem and answer    
    word_types = ['NN', 'NNS', 'NNP', 'NNPS', 'RB', 'RBR', 'RBS', 'VB', 'VBG', 'VBN', 'VBP', 'VBZ']
    stem_token = sent_tokenize(question.stem)
    stem_words = []
    for i in stem_token:
        wordsList = nltk.word_tokenize(i)
        wordsList = [w for w in wordsList if not w in stop_words]
        tagged = nltk.pos_tag(wordsList)
        for t in tagged:
            if t[1] in word_types:
                stem_words.append(t[0].lower())
                
    ans_token = sent_tokenize(question.correct_option)
    ans_words = []
    for i in ans_token:
        wordsList = nltk.word_tokenize(i)
        wordsList = [w for w in wordsList if not w in stop_words]
        tagged = nltk.pos_tag(wordsList)
        for t in tagged:
            if t[1] in word_types:
                ans_words.append(t[0].lower())
    
    if any(x in stem_words for x in ans_words):
        return False

    
    return True

def get_lemma_nouns(text):
    all_nouns = []
    tokenized = sent_tokenize(text)
    
    for i in tokenized:

        # Word tokenizers is used to find the words and punctuation in a string
        wordsList = nltk.word_tokenize(i)

        # removing stop words from wordList
        wordsList = [w for w in wordsList if not w in stop_words]

        #  Using a Tagger. Which is part-of-speech tagger or POS-tagger.
        tagged = nltk.pos_tag(wordsList)
        
        # Add any nouns to this list
        for t in tagged:
            if t[1] in nouns:
                all_nouns.append(t[0].lower())
    
    lemmatized_nouns = []
    for n in all_nouns:
        lemmatized_word = lemmatizer.lemmatize(n, pos="n")
        lemmatized_nouns.append(lemmatized_word.lower())
    
    return lemmatized_nouns

In [314]:
#Readability score (perhaps BLEU and shit, maybe the BERT-based one?)
def unfocused_stem(question):
    contains_question = False
    doc = nlp(question.stem)
    for sent in doc.sents:
        if is_question(sent.text.strip()):
            contains_question = True
            
    if not contains_question:
        print("--- Question stem does not contain a question")
        
    return contains_question

#https://stackoverflow.com/questions/72548388/is-there-a-machine-learning-or-nlp-model-to-separate-questions-and-answers-in-ra
#https://stackoverflow.com/questions/4083060/determine-if-a-sentence-is-an-inquiry
def is_question(sent):
    d = nlp(sent)
    token = d[0] # gets the first token in a sentence
    if token.pos_ == "VERB" and token.dep_ == "ROOT": # checks if the first token is a verb and root or not
        return True
    for token in d: # loops through the sentence and checks for WH tokens
        if token.tag_ == "WDT" or token.tag_ == "WP" or token.tag_ == "WP$" or token.tag_ == "WRB" or token.text == '?':
            return True
    return  False

In [315]:
def clean_string(string):
    # remove whitespace
    cleaned_string = string.strip()
    
    # remove punctuation
    cleaned_string = re.sub(r'[^\w\s]', '', cleaned_string)
    
    # remove list notation
    cleaned_string = re.sub(r'\b(i|ii|iii|iv|v|vi|vii|viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx)\b', '', cleaned_string)
    cleaned_string = re.sub(r'\b(A|B|C|D|E|F)\b', '', cleaned_string)
    print('cleaned_string: ', cleaned_string)
    return cleaned_string

#Check for commas in the answer choices? Repeated words between answer choices?
#Also similar to the convergence cue one
#Now it does a good job picking up on teh 1. or ii/iii/iv or (A) (B) (C) style questions, how can we do that?
def complex_k_type(question):
    #if the answer options share the same words between one another 
    #and there are commas present
    #then it's k type
    #also the stem says "Select all that apply" or "Select all" in general?
    
    options = question.options.copy()
    options.remove(question.correct_option)
    if len(options) < 3:
        return True 
    
    #check if the options contain a comma
    contain_a_comma = 0
    for opt in options:
        if ',' in opt:
            contain_a_comma += 1
    contain_a_comma = contain_a_comma == len(options)
    
    lemma_nouns_answ = get_lemma_nouns(question.correct_option)
    lemma_nouns_options = [get_lemma_nouns(options[0]), get_lemma_nouns(options[1]), get_lemma_nouns(options[2])]
    
    options_that_share_noun = 0
    for lno in lemma_nouns_options:   
        repeating_nouns = list(set(lno).intersection(lemma_nouns_answ))
        if (len(repeating_nouns) > 0) and (len(lno) > 0):
            options_that_share_noun += 1
    
    #Options share a key word, there are multiple nouns in the options, and they have a comma
    #suggesting it might be a k-type question
    if options_that_share_noun > 0 and contain_a_comma:
        print("--- This might be a K-type question")
        return False
    
    #After removing any list notation in the answer choices, see if they contain the same words
    cleaned_options = []
    for opt in options:
        cleaned_options.append(clean_string(opt))

    options_set_list = [set(i.split()) for i in cleaned_options]
    if options_set_list[0] == options_set_list[0] and options_set_list[0] == options_set_list[1] and options_set_list[0] == options_set_list[2]:
        return False
    
    return True

In [316]:
#If verb exists in answer choice, ensure it's the same tense as verb in other options
# we want the stem to be the same, but as long as all the answers are the same, then it's fine, to avoid false positive

# (Done in logical cue) NER, if the answer choice contains a specific type of noun, the other options should likely contain that
#VB  --  verb, base form
# VBD  --  verb, past tense
# VBG  --  verb, gerund or present participle
# VBN  --  verb, past participle
# VBP  --  verb, non-3rd person singular present
# VBZ  --  verb, 3rd person singular present

def grammatical_cues_in_stem(question):
    stem_tense = get_verb_tense(question.stem)
    answer_tense = get_verb_tense(question.correct_option)
    
    options = question.options.copy()
    options.remove(question.correct_option)
    for opt in options:
        opt_tense = get_verb_tense(opt)
        if opt_tense != 'none' and answer_tense is not opt_tense:
            print("--- verb tense doesn't align between answer other options")
            return False
        
    return True
    
def get_verb_tense(text):
    doc = nlp(text)
    for token in doc:
        if token.pos_ == 'VERB':
            if token.tag_ in ['VBP', 'VBZ']:
                return 'present'
            elif token.tag_ in ['VBD', 'VBN']:
                return 'past'
            else:
                return 'other'
    return 'none'

In [317]:
#If answer choices are numeric, sort them, compare to current order
#Otherwise how the hell do we know this one..? No need to alphabetize imo (unless it’s like one word)
def lost_sequence(question):
    options = question.options.copy()
    
    opts = []
    for opt in options:
        opt = re.sub(r'[$%°FC,]', '', opt)
        opts.append(opt)
        try:
            float(opt)
        except ValueError:
            return True
        
    float_options = [float(x) for x in opts]    
    sorted_options = sorted(float_options)
    if sorted_options == float_options:
        #Numeric options are sorted
        return True
    else:
        print('--- Options are numeric and not sorted')
        return False

In [318]:
#This one might need POS tagging too, because an absolute term isn't a deal breaker in some cases
# i.e. "Why does Hydrogen never bond with...?" 
def vague_terms(question):
    vagues = ["often", "sometimes", "rarely", "typically", "usually", "normally", "generally", "nearly", "approximately", "more or less", "somewhat"]
    for opt in question.options:
        if any(word in opt for word in vagues):
            print('--- vague word in question stem')
            return False
    
    return True

In [319]:
#QA model confirm correct answer is correct → (make the QA model do it’s best guess, 
#but maybe use like gpt-3 AND some QA model?)
#I think we can say “all answers are correct, the QA Model was great, but in the future something more robust 
#like ChatGPT should be utilized? As we input all the questions and they were right.
completion = None
def more_than_one_correct(question):
    if not question.options[2]:
        question.options[2] = 'None'
        
    if not question.options[3]:
        question.options[3] = 'None'
    
    # Define the prompt
    prompt = """
    Answer the multiple-choice question below by responding with A, B, C, or D.
    
    {}

    A) {}
    B) {}
    C) {}
    D) {}
    """.format(question.stem, question.options[0], question.options[1], question.options[2], question.options[3])

    # Generate a response
    completion = openai.Completion.create(engine=model_engine, prompt=prompt, max_tokens=1024, n=1,stop=None,temperature=0.5)

    # Print the response
    try:
        #cleaned_response = completion.choices[0].text.split(')')[1].strip()
        cleaned_response = completion.choices[0].text.split(')')[0].strip()
        if (cleaned_response == 'A'):
            cleaned_response = question.options[0]
        elif (cleaned_response == 'B'):
            cleaned_response = question.options[1]
        elif (cleaned_response == 'C'):
            cleaned_response = question.options[2]
        elif (cleaned_response == 'D'):
            cleaned_response = question.options[3]
    except: 
        print('error in GPT3: ', completion)
    
    if cleaned_response == question.correct_option:
        return True
    else:
        print('--- GPT-3 believes the answer is incorrect: ', cleaned_response, ' ', question.correct_option)
        return False

In [320]:
#This one might need POS tagging too, because an negative term isn't a deal breaker in some cases
# i.e. "If there is no oxygen present when the chemical reacts, then what does...?" 
#Sentiment analysis for this? nlp determine if something is negatively worded
#Now it checks for "not" and "incorrect" and "false" paired with "Which" in the same sentence? Perhaps split by period
def negative_worded_stem(question):
    negatives = ["no", "none", "never", "without", "exclude", "avoid", "deny", "refuse", "oppose", "dispute"]
    for opt in question.stem:
        if any(word in opt for word in negatives):
            print('--- absolute word in question stem')
            return False
    
    #Check for neatively worded stem too.
    for sent in question.stem.split('.'):
        sent = sent.lower()
        #What is common too, but might have too many falses?
        if 'which' in sent and ('false' in sent or 'not' in sent or 'incorrect' in sent or 'except' in sent) or \
        'what' in sent and ('false' in sent or 'not' in sent or 'incorrect' in sent or 'except' in sent):
            return False    
    
    
    return True

## Peerwise Biochemistry Data - 400 Questions

In [408]:
qs = pd.read_csv('peerwise_questions.csv')
# filtered_df = qs.loc[(qs['avg_rating'] > 3.49) & (qs['total_ratings'] > 9) & (qs['numAlts'] == 4)]
# filtered_df = filtered_df[~filtered_df['question'].str.casefold().str.contains("img")]
columns_to_clean = ['question', 'altA', 'altB', 'altC', 'altD']

def clean_peerwise_data(cdf):
    for cName in columns_to_clean:
        cdf[cName] = cdf[cName].str.replace(r'<[^<>]*>', ' ', regex=True)
        cdf[cName] = cdf[cName].apply(lambda x: re.sub("&nbsp;",  "", x))
        cdf[cName] = cdf[cName].str.replace(' +', ' ', regex=True)
        cdf[cName] = cdf[cName].str.strip()

    peerwise_student_questions = []
    for index, row in cdf.iterrows():
        question = MultipleChoiceQuestion(
            stem=row['question'],
            options=[row['altA'], row['altB'], row['altC'], row['altD']],
            correct_option= row['alt' + row['answer'].strip()],
            qid = row['total_response'],
            courseid = row['avg_difficulty'],
            quality = row['avg_rating']
        )
        if not question.stem or not question.options[0] or not question.options[1] or not question.options[2] or not question.options[3]:
            print('removing empty option question')
        else:
            peerwise_student_questions.append(question)
    
    selected_items = random.sample(peerwise_student_questions, 400)
    return selected_items


# # biochem_lowest_df = qs.loc[(qs['avg_rating'] < 2.0) & (qs['avg_rating'] > 1.49) & (qs['total_ratings'] > 9) & (qs['numAlts'] == 4)]
# # biochem_lowest = clean_peerwise_data(biochem_lowest_df)

# # biochem_low_df = qs.loc[(qs['avg_rating'] < 2.5) & (qs['avg_rating'] > 1.99) & (qs['total_ratings'] > 19) & (qs['numAlts'] == 4)]
# # biochem_low = clean_peerwise_data(biochem_low_df)

# # biochem_mid_df = qs.loc[(qs['avg_rating'] < 3.0) & (qs['avg_rating'] > 2.49) & (qs['total_ratings'] > 19) & (qs['numAlts'] == 4)]
# # biochem_mid = clean_peerwise_data(biochem_mid_df)

# # biochem_high_df = qs.loc[(qs['avg_rating'] < 3.5) & (qs['avg_rating'] > 2.99) & (qs['total_ratings'] > 19) & (qs['numAlts'] == 4)]
# # biochem_high = clean_peerwise_data(biochem_high_df)

# # biochem_highest_df = qs.loc[(qs['avg_rating'] > 3.49) & (qs['total_ratings'] > 6) & (qs['numAlts'] == 4)]
# # biochem_highest = clean_peerwise_data(biochem_highest_df)

# # peerwise_student_questions = biochem_lowest + biochem_low + biochem_mid + biochem_high + biochem_highest    
# # print('length: ', len(peerwise_student_questions))

take2 = qs.loc[(qs['total_response'] > 29) & (qs['numAlts'] == 4)]
biochem_take2 = clean_peerwise_data(take2)

peerwise_student_questions = biochem_take2

rows = []
i = 0
for q in peerwise_student_questions:
    r = [ambiguous_unclear_information(q),
        implausible_distractors(q),
        none_of_the_above(q),
        longest_answer_correct(q),
        gratuitous_information_in_stem(q),
        true_or_false(q),
        avoid_convergence_cues(q),
        avoid_logical_cues(q),
        all_of_the_above(q),
        fill_in_the_blank(q),
        absolute_terms(q),
        word_repeats_in_stem_and_correct_answer(q),
        unfocused_stem(q),
        complex_k_type(q),
        grammatical_cues_in_stem(q),
        lost_sequence(q),
        vague_terms(q),
        #more_than_one_correct(q),
        negative_worded_stem(q),
        q.qid,
        q.courseid,
        q.quality]
    rows.append(r)
    print('i: ', i)
    i += 1
    
columns = [
    'ambiguous_unclear_information',
    'implausible_distractors',
    'none_of_the_above',
    'longest_answer_correct',
    'gratuitous_information_in_stem',
    'true_or_false',
    'avoid_convergence_cues',
    'avoid_logical_cues',
    'all_of_the_above',
    'fill_in_the_blank',
    'absolute_terms',
    'word_repeats_in_stem_and_correct_answer',
    'unfocused_stem',
    'complex_k_type',
    'grammatical_cues_in_stem',
    'lost_sequence',
    'vague_terms',
    #'more_than_one_correct',
    'negative_worded_stem',
    'qid',
    'courseid',
    'quality'
]

peerwise_results = pd.DataFrame(rows, columns=columns)
peerwise_results.to_csv("peerwise_400diffv2_results_auto_iwf.csv")

#Print the number of "Falses", which correspond to IWF, found the table across each criteria
peerwise_400diffv2_results = peerwise_results.copy()
peerwise_400diffv2_results = peerwise_400diffv2_results.drop(columns=['qid', 'courseid', 'quality'])
for c in columns[:len(columns)-3]:
    print(c, ': ', (~peerwise_400diffv2_results[c]).values.sum())

C:\Users\Steven\AppData\Local\Temp\ipykernel_15452\1702380357.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf[cName] = cdf[cName].str.replace(r'<[^<>]*>', ' ', regex=True)
C:\Users\Steven\AppData\Local\Temp\ipykernel_15452\1702380357.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf[cName] = cdf[cName].apply(lambda x: re.sub("&nbsp;",  "", x))
C:\Users\Steven\AppData\Local\Temp\ipykernel_15452\1702380357.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
removing empty option question
opt_entity:  ETC coupling
lemma_nouns_opt:  ['etc', 'coupling']
ans_entity:  ATP production in type I

Question: C1-17 What are the main characteristics of a rate limiting step, or enzyme, of metabolism?
 Slow, [S] &lt;&lt; Km and saturated with substrate
Fast, catalyses a reversible reaction and is working at Vmax
Slow, catalyses a irreversible reaction and is working at Vmax
Slow, operating under low [S] and catalyses an irreversible reaction
Correct option: Slow, catalyses a irreversible reaction and is working at Vmax
Quality: 2.82
repeating_nouns_synonyms:  ['slow']
--- we have a synonym of the answer being used in other answer choices, but not all of them:  slow
--- This might be a K-type question
--- verb tense doesn't align between answer other options
i:  6
Question: In our daily nitrogen balance:
 The amount of output of nitrogen is less than the input
Approximately 16.5g of nitrogen is excreted through urea everyday
Approximately 16.5g of nitrogen is excreted through urea and faeces everyday
The amount of input of nitrogen is the same as the output
Correct option: The amount 

opt_entity:  DNA Polymerase 1
lemma_nouns_opt:  ['dna', 'polymerase']
ans_entity:  dNTPs
lemma_nouns_ans:  ['dntps']
Question: Which of the following PCR components is used in the Sanger method of DNA Sequencing?
 DNA Polymerase 1
Single-stranded DNA Molecule
A pair of DNA primers
dNTPs
Correct option: dNTPs
Quality: 2.8
--- distractor not similar enough
dNTPs 		 DNA Polymerase 1 		 Score: 0.0358
Question: Which of the following PCR components is used in the Sanger method of DNA Sequencing?
 DNA Polymerase 1
Single-stranded DNA Molecule
A pair of DNA primers
dNTPs
Correct option: dNTPs
Quality: 2.8
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
cleaned_string:  DNA Polymerase 1
cleaned_string:  Singlestranded DNA Molecule
cleaned_string:   pair of DNA primers
--- verb tense doesn't align between answer other options
i:  13
opt_entity:  Hydrophobic
lemma_nouns_opt:  ['hydrophobic']
ans_entity:  All of above
lemma_nouns_ans:  []
Question: The fo

--- verb tense doesn't align between answer other options
i:  22
Question: C5-21 Which of the following regarding cholesterol's role in a cell membrane is incorrect ?
 Adding cholesterol to a membrane with too many unsaturated fatty acids will decrease its fluidity.
Cholesterol acts as a membrane fluidity 'buffer'.
Adding cholesterol to a membrane with lots of saturated fatty acids will increase its solidity.
Cholesterol helps keep the membrane impermeable to some water soluble molecules.
Correct option: Adding cholesterol to a membrane with lots of saturated fatty acids will increase its solidity.
Quality: 2.86
repeating_nouns_synonyms:  ['acid', 'cholesterol']
--- we have a synonym of the answer being used in other answer choices, but not all of them:  acid
cleaned_string:  Adding cholesterol to a membrane with too many unsaturated fatty acids will decrease its fluidity
cleaned_string:  Cholesterol acts as a membrane fluidity buffer
cleaned_string:  Cholesterol helps keep the membran

...

Question: Why does NOT having the 5' to 3' exonuclease make Klenow so useful in molecular biology?
 All the 5' to 3' exonuclease chewing up ahead of replication makes it too slow to be practical
The 5' to 3' exonuclease can act unpredictably in vitro leading to lots of mistakes
The &lsquo;futile&rsquo; cycle of nucleotide incorporation and removal wastes tons of dNTP substrate
Removal of the 5&rsquo; to 3&rsquo; exonuclease makes the polymerase stay on track better
Correct option: The &lsquo;futile&rsquo; cycle of nucleotide incorporation and removal wastes tons of dNTP substrate
Quality: 2.42
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  ['removal']
--- we have a synonym of the answer being used in other answer choices, but not all of them:  removal
--- The answer entity is not found in any other options:  ORG
cleaned_string:  All the 5 to 3 exonuclease chewing up ahead of replication makes it too slow to be practical
cleaned_string:  The 5 to 

Question: C1-01: Sally is a race driver who is about to compete in the F1 tournament. Sally believes that a full tank is key to winning, and has invented a pipe that transports petrol into the engine to keep it topped up. Howard is Sally's main competition who drives the same car. Howard has a different tactic, and has implemented a state of the art internal combustion engine which combusts fuel and oxygen to move pistons 2x faster than those of regular combustion engines. Using the information above and your knowledge of metabolism, select the most correct option from the statements below.
 It is likely that Sally will win the tournament given that her petrol tank is always full. Fuel levels are primarily responsible for the rate of car acceleration, and can be likened to the concentration of fatty acids and carbohydrates in our bodies, where a greater concentration leads to a faster rate of metabolism
The pistons in Howard's car can be likened to the spinning protein past the F0 chan

Question: Mr. Green, 60, who has been told that he needs to lose weight and in particular control the cholesterol level in his blood, otherwise his health will be threatened. Which way is the best to help Mr. Green?
 Immediately become a vegan, no more meat.
Reduce the amount of full fat milk, butter and sausages in meals.
Include food that contains high amount of phytosterols to inhibit cholesterol, such as walnuts, vegetable, fruits and legumes, in total at least 1 kg per day to reduce cholesterol as fast as possible.
Most of the cholesterol is produced in the body, not from the intake of food, he just needs to change dietary habit and do more exercise, but he can keep smoking.
Correct option: Reduce the amount of full fat milk, butter and sausages in meals.
Quality: 2.42
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  ['amount']
--- we have a synonym of the answer being used in other answer choices, but not all of them:  amount
--- This might be a K-type question
--- verb t

Question: If a mutation caused a cell to rapidly reduce its rate of functioning, ADP stores would:
 decrease as they are turned into stable ATP
remain stagnant as the cell no longer functions
decrease as they are quickly used by the body for energy
increase to accomodate for increased ADP usage by other cells
Correct option: decrease as they are turned into stable ATP
Quality: 2.51
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  ['decrease']
--- we have a synonym of the answer being used in other answer choices, but not all of them:  decrease
--- The answer entity is not found in any other options:  ORG
--- Question stem does not contain a question
cleaned_string:  remain stagnant as the cell no longer functions
cleaned_string:  decrease as they are quickly used by the body for energy
cleaned_string:  increase to accomodate for increased ADP usage by other cells
--- verb tense doesn't align between answer other options
i:  388
Question: C2-25 Kevin is waiting for his friend at

Question: Scientists collected some blood samples from individuals who have been starving for more than 30 days to measure the plasma level of glucose, fatty acids and ketone bodies as shown in the graph below. Which of the following is NOT the reason whystarving for more than 30 daysis unsustainable?
 Other organs still need constant glucose supply as their main energy source from proteolysis
Ketone bodies can spontaneous decarboxylate acetoacetate to acetone which is a useless molecule
The break down of protein can only generate 25% glucose (1g protein= 1/4 g glucose)
Depletion of protein from tissue compromised immune system
Correct option: The break down of protein can only generate 25% glucose (1g protein= 1/4 g glucose)
Quality: 2.76
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  ['protein']
--- we have a synonym of the answer being used in other answer choices, but not all of them:  protein
--- absolute word in question stem
cleaned_string

In [398]:
# Calculate the correlation matrix
corr_matrix = peerwise_results.corr()

# Number of tests to conduct
num_tests = 5

# Significance level (alpha)
alpha = 0.05

# Bonferroni corrected significance level
bonferroni_alpha = alpha / num_tests

# Print the correlation matrix
print(f"Bonferroni corrected alpha: {bonferroni_alpha}")
corr_matrix

Bonferroni corrected alpha: 0.01


,ambiguous_unclear_information,implausible_distractors,none_of_the_above,longest_answer_correct,gratuitous_information_in_stem,true_or_false,avoid_convergence_cues,avoid_logical_cues,all_of_the_above,fill_in_the_blank,...,unfocused_stem,complex_k_type,grammatical_cues_in_stem,lost_sequence,vague_terms,more_than_one_correct,negative_worded_stem,qid,courseid,quality
ambiguous_unclear_information,1.000000,0.027856,0.093918,-0.009219,0.002034,0.038109,-0.049504,0.117052,-0.023956,0.002034,...,-0.043890,-0.041221,0.052327,NaN,-0.054114,0.000206,0.032566,-0.040438,-0.042121,0.109842
implausible_distractors,0.027856,1.000000,-0.040561,0.105541,0.045466,-0.029022,-0.204681,-0.084866,-0.023300,0.005844,...,-0.009519,-0.147433,0.105275,NaN,0.001350,-0.067654,0.083610,-0.014047,-0.018423,0.175869
none_of_the_above,0.093918,-0.040561,1.000000,0.060444,-0.027558,0.073101,0.046295,-0.031863,-0.008748,-0.027558,...,0.109886,-0.000275,0.033183,NaN,-0.019760,0.003086,-0.011718,-0.090980,-0.086918,-0.026564
longest_answer_correct,-0.009219,0.105541,0.060444,1.000000,-0.053874,0.196816,-0.055594,-0.016988,-0.017102,-0.053874,...,-0.034209,-0.108213,0.050220,NaN,-0.038631,-0.015905,0.149126,-0.034584,-0.034840,0.056293
gratuitous_information_in_stem,0.002034,0.045466,-0.027558,-0.053874,1.000000,-0.087275,0.038447,-0.009104,-0.015830,0.005387,...,-0.068063,0.088680,-0.096994,NaN,-0.035759,-0.029686,0.032171,0.197406,0.197992,-0.115811
true_or_false,0.038109,-0.029022,0.073101,0.196816,-0.087275,1.000000,0.050957,-0.039983,-0.027704,-0.052607,...,-0.092676,-0.096315,0.174060,NaN,-0.015350,0.017150,0.349515,-0.054649,-0.051627,0.044058
avoid_convergence_cues,-0.049504,-0.204681,0.046295,-0.055594,0.038447,0.050957,1.000000,-0.051600,0.051447,-0.011002,...,0.013766,-0.167382,0.184300,NaN,0.116214,-0.154928,-0.048331,-0.016716,-0.014276,-0.097631
avoid_logical_cues,0.117052,-0.084866,-0.031863,-0.016988,-0.009104,-0.039983,-0.051600,1.000000,-0.018303,-0.009104,...,-0.078696,-0.032843,-0.006469,NaN,-0.041345,-0.045199,0.099135,0.103182,0.106941,0.032030
all_of_the_above,-0.023956,-0.023300,-0.008748,-0.017102,-0.015830,-0.027704,0.051447,-0.018303,1.000000,-0.015830,...,-0.021606,-0.031797,0.066421,NaN,-0.011351,0.001773,-0.035997,0.125361,0.127883,-0.006246
fill_in_the_blank,0.002034,0.005844,-0.027558,-0.053874,0.005387,-0.052607,-0.011002,-0.009104,-0.015830,1.000000,...,0.016226,-0.005744,-0.144107,NaN,-0.035759,-0.100228,0.003057,0.197839,0.202253,-0.019416


## Peerwise Biochemistry - 50 Questions 

In [337]:
# #Maybe grab 10 from each: 1.5-1.99, 2.0-2.49, 2.50-2.99, 3.00-3.49, 3.50-3.99
# #It's fine in the paper to say these are higher quality and came from a different system, comparing OLI and PeerWise is not 
# #the point, it's just that even when such a system is used, problems persist with student-authored questions.
# qs = pd.read_csv('peerwise_questions.csv')
# def clean_df(fdf):
#     fdf = fdf[~fdf['question'].str.casefold().str.contains("img")]
#     columns_to_clean = ['question', 'altA', 'altB', 'altC', 'altD']

#     for cName in columns_to_clean:
#         fdf[cName] = fdf[cName].str.replace(r'<[^<>]*>', ' ', regex=True)
#         fdf[cName] = fdf[cName].apply(lambda x: re.sub("&nbsp;",  "", x))
#         fdf[cName] = fdf[cName].str.replace(' +', ' ', regex=True)
#         fdf[cName] = fdf[cName].str.strip()
        
#         fdf[fdf[cName].apply(lambda x: len(x.strip()) > 2)]
#         fdf[cName].replace('', np.nan, inplace=True)
#     fdf.dropna(subset=['altA'], inplace=True)
#     return fdf

# biochem_lowest = qs.loc[(qs['avg_rating'] < 2.0) & (qs['avg_rating'] > 1.49) & (qs['total_ratings'] > 19) & (qs['numAlts'] == 4)]
# biochem_lowest = clean_df(biochem_lowest)

# biochem_low = qs.loc[(qs['avg_rating'] < 2.5) & (qs['avg_rating'] > 1.99) & (qs['total_ratings'] > 19) & (qs['numAlts'] == 4)]
# biochem_low = clean_df(biochem_low)

# biochem_mid = qs.loc[(qs['avg_rating'] < 3.0) & (qs['avg_rating'] > 2.49) & (qs['total_ratings'] > 19) & (qs['numAlts'] == 4)]
# biochem_mid = clean_df(biochem_mid)

# biochem_high = qs.loc[(qs['avg_rating'] < 3.5) & (qs['avg_rating'] > 2.99) & (qs['total_ratings'] > 19) & (qs['numAlts'] == 4)]
# biochem_high = clean_df(biochem_high)

# biochem_highest = qs.loc[(qs['avg_rating'] > 3.49) & (qs['total_ratings'] > 19) & (qs['numAlts'] == 4)]
# biochem_highest = clean_df(biochem_highest)

# sampled_dfs = [biochem_lowest.sample(n=10), biochem_low.sample(n=10), biochem_mid.sample(n=10), biochem_high.sample(n=10), biochem_highest.sample(n=10)]
# fifty_biochem_questions = pd.concat(sampled_dfs, axis=0).reset_index()

#This uses the ones tagged by IWF
# qs = pd.read_csv('peerwise_50_questions.csv')

# peerwise_biochem_questions = []
# for index, row in qs.iterrows():
#     question = MultipleChoiceQuestion(
#         stem=row['question'],
#         options=[row['altA'], row['altB'], row['altC'], row['altD']],
#         correct_option= row['alt' + row['answer'].strip()],
#         qid = row['student'],
#         courseid = row['course'],
#         quality = row['course']
#     )
#     peerwise_biochem_questions.append(question)

# rows = []
# i = 0
# for q in peerwise_biochem_questions:
#     r = [ambiguous_unclear_information(q),
#         implausible_distractors(q),
#         none_of_the_above(q),
#         longest_answer_correct(q),
#         gratuitous_information_in_stem(q),
#         true_or_false(q),
#         avoid_convergence_cues(q),
#         avoid_logical_cues(q),
#         all_of_the_above(q),
#         fill_in_the_blank(q),
#         absolute_terms(q),
#         word_repeats_in_stem_and_correct_answer(q),
#         unfocused_stem(q),
#         complex_k_type(q),
#         grammatical_cues_in_stem(q),
#         lost_sequence(q),
#         vague_terms(q),
#         more_than_one_correct(q),
#         negative_worded_stem(q),
#         q.qid,
#         q.courseid,
#         q.quality]
#     rows.append(r)
#     i += 1
    
# columns = [
#     'ambiguous_unclear_information',
#     'implausible_distractors',
#     'none_of_the_above',
#     'longest_answer_correct',
#     'gratuitous_information_in_stem',
#     'true_or_false',
#     'avoid_convergence_cues',
#     'avoid_logical_cues',
#     'all_of_the_above',
#     'fill_in_the_blank',
#     'absolute_terms',
#     'word_repeats_in_stem_and_correct_answer',
#     'unfocused_stem',
#     'complex_k_type',
#     'grammatical_cues_in_stem',
#     'lost_sequence',
#     'vague_terms',
#     'more_than_one_correct',
#     'negative_worded_stem',
#     'qid',
#     'courseid',
#     'quality'
# ]

# peerwise_biochem_results = pd.DataFrame(rows, columns=columns)
# peerwise_biochem_results.to_csv("biochem_results_auto_iwf.csv")

# #Print the number of "Falses", which correspond to IWF, found the table across each criteria
# for c in columns:
#     print(c, ': ', (~peerwise_biochem_results[c]).values.sum())

Question: Which of the following is not a characteristic of an emulsion forming molecule?
 They are amphiphilic.
They act as detergents.
They form dricelles.
They are amphipatic.
Correct option: They form dricelles.
Quality: 1.6
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
--- The answer entity is not found in any other options:  ORG
cleaned_string:  They are amphiphilic
cleaned_string:  They act as detergents
cleaned_string:  They are amphipatic
--- GPT-3 believes the answer is incorrect:  They are amphipatic.   They form dricelles.
Question: At what temperature is the DNA double helix denatured in PCR?
 54 o C
95 o C
60 o C
75 o C
Correct option: 95 o C
Quality: 1.81
repeating_nouns_synonyms:  ['c']
repeating_nouns_synonyms:  ['c']
repeating_nouns_synonyms:  ['c']
cleaned_string:  54 o 
cleaned_string:  60 o 
cleaned_string:  75 o 
opt_entity:  Growth factors
lemma_nouns_opt:  ['growth', 'factor']
ans_entity:  DNA-dependent RNA polymerase


--- GPT-3 believes the answer is incorrect:  It is uncalled for. Because endogenous antioxidant will be enough to balance the free-radical produced from exercise.   It is uncalled for. Because exercise might cause cells to produce more free-radical, but it also will produce more endogenous antioxidant, and with natural dietary antioxidant intake (e.g. in orange), there won&rsquo;t be ill effect on the body.
opt_entity:  Initiation
lemma_nouns_opt:  ['initiation']
ans_entity:  Relocation
lemma_nouns_ans:  ['relocation']
Question: What is a process NOT used inEukaryotic translation?
 Initiation
Elongation
Relocation
Termination
Correct option: Relocation
Quality: 1.94
--- distractor not similar enough
Relocation 		 Initiation 		 Score: 0.1351
Question: What is a process NOT used inEukaryotic translation?
 Initiation
Elongation
Relocation
Termination
Correct option: Relocation
Quality: 1.94
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
cleaned_s

--- GPT-3 believes the answer is incorrect:  All of the above   Uses very low amounts of DNA
Question: The components of a computer including the devices (such as a printer), hard-drive and RAM storage can be said to be analogous to the information storage components in a cell; the DNA, RNA and protein. Given this analogy, and thinking about which parts of the computer these cell components are similar to, choose the most correct answer from the following.
 DNA is like the hard-drive of the computer as different computers, like every different cell in your body, contains different information stored in their DNA so they are able to perform the different purposes needed.
While it is impossible for information to be sent directly from the hard-drive to devices such as printer, this is a point the metaphor breaks as it is possible for information to be sent from DNA directly to protein.
Information can never flow from DNA to RNA, only from DNA to RNA in the same sense that information in 

Question: You are trying to create a standard curve, but accidentally take your measurements using a cuvette with a 2cm light path instead of 1cm. What impact does this increased light path have on the equation of your resulting standard curve?
 The gradient of the curve becomes steeper, or greater, than you expected.
The gradient of the curve becomes flatter, or less, than you expected.
The gradient of the curve becomes inverted (i.e. negative/ slopes downward instead of upward).
No effect.
Correct option: The gradient of the curve becomes steeper, or greater, than you expected.
Quality: 2.36
repeating_nouns_synonyms:  ['gradient', 'curve']
--- we have a synonym of the answer being used in other answer choices, but not all of them:  gradient
cleaned_string:  The gradient of the curve becomes flatter or less than you expected
cleaned_string:  The gradient of the curve becomes inverted ie negative slopes downward instead of upward
cleaned_string:  No effect
--- GPT-3 believes the answer

--- GPT-3 believes the answer is incorrect:  None of the above   The overall ATP utilization is greater in gluconeogenesis than in glycolysis
Question: Which force contributes the most to stabilizing the DNA double-helix?
 a) Hydrophobic bases are found in the interior of the helix where each base- pair is stabilized by the same number of hydrogen bonds.
b) Hydrophilic sugar-phosphate groups are found on the exterior of the helix where they can interact with water.
c) Non-covalent N-glycosidic bonds can form between nitrogenous bases in opposite strands in the helix.
d) Covalent base-stacking interactions can occur between adjacent bases within the same strand in the helix.
Correct option: b) Hydrophilic sugar-phosphate groups are found on the exterior of the helix where they can interact with water.
Quality: 2.77
repeating_nouns_synonyms:  ['helix']
repeating_nouns_synonyms:  ['helix']
repeating_nouns_synonyms:  ['helix']
cleaned_string:  a Hydrophobic bases are found in the interior 

--- GPT-3 believes the answer is incorrect:  I and IV are true II and III are false   III and IV are true I and II are false
Question: Which of the following is correct in describing the role of NAD+ in ATP synthesis?
 The pumping of protons powered by NAD+ allows the uncoupling protein to dissipate the proton gradient, hence resulting in ATP synthesis.
NAD+ breaks down fats and carbohydrates in the body, and that then provides the electrons for the electron transport chain, hence resulting in ATP synthesis.
NAD+ is reduced to NADH via the breakdown of fats and carbohydrates in the body, and that then provides the electrons for the electron transport chain, hence resulting in ATP synthesis.
NAD+ binds to the uncoupling protein to inhibit ATP synthesis
Correct option: NAD+ is reduced to NADH via the breakdown of fats and carbohydrates in the body, and that then provides the electrons for the electron transport chain, hence resulting in ATP synthesis.
Quality: 3.0
repeating_nouns_synonym

--- GPT-3 believes the answer is incorrect:  The main regulator, phosphofructokinase, is strongly affected by ATP and ADP levels due to the energy charge fluctuations in the cells. However, AMP doesn&rsquo;t affect phosphofructokinase as this enzyme cannot recognize the single phosphate attached to the adenine.   Metabolic pathways are very sensitive to the energy charge of a cell (i.e. ATP levels in the cell) such that a drop in ATP levels would cause hexokinase, phosphofructokinase and pyruvate kinase to respond dramatically.
opt_entity:  I don't know
lemma_nouns_opt:  []
ans_entity:  The theory does not apply because metabolic performance DOES NOT increase with smaller meals i.e. metabolism is not regulated by fuel availability( unlike fire).
lemma_nouns_ans:  ['theory', 'performance', 'doe', 'not', 'meal', 'metabolism', 'availability', 'fire']
Question: Metaphors for Metabolism. You&rsquo;re planning a long car trip in a 4WD, and your friend strongly believes that his car &ldquo;wo

--- This might be a K-type question
--- verb tense doesn't align between answer other options
--- GPT-3 believes the answer is incorrect:  If Derek decides that the weather is too hot and decides to lie down on the tennis court instead of exercising, then the decrease in work will cause ATP consumption to DECREASE, rate of proton pumping to DECREASE, and oxygen consumption will DECREASE.   If Derek had a gigantic feast of KFC in the middle of his tennis session and there is now a readily available amount of fuel for his body to metabolise, then the rate of ATP synthesis will INCREASE, rate of proton pumping will INCREASE and oxygen consumption will also INCREASE.
Question: You want the initial rate of the reaction to be about 0.75 absorbance units per minute. What is this rate of change expressed in moles per minute in the well? Extinction Coefficient: 15 mM -1 cm -1 ,200uL solution with cuvette length of 1cm.
 50 nmol min -1
10 nmol min -1
5 nmol min- 1
15nmol min -1
Correct option: 1

TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

## OLI Chemistry Data - 50 Questions

In [323]:
# qs = pd.read_csv('oli_chemistry_questions.csv')
# qs = qs.fillna('')

# oli_chemistry_student_questions = []
# for index, row in qs.iterrows():
#     question = MultipleChoiceQuestion(
#         stem=row['question'],
#         options=[row['altA'], row['altB'], row['altC'], row['altD']],
#         correct_option= row['altA']
#     )
#     oli_chemistry_student_questions.append(question)

# rows = []
# i = 0
# for q in oli_chemistry_student_questions:
#     r = [ambiguous_unclear_information(q),
#         implausible_distractors(q),
#         none_of_the_above(q),
#         longest_answer_correct(q),
#         gratuitous_information_in_stem(q),
#         true_or_false(q),
#         avoid_convergence_cues(q),
#         avoid_logical_cues(q),
#         all_of_the_above(q),
#         fill_in_the_blank(q),
#         absolute_terms(q),
#         word_repeats_in_stem_and_correct_answer(q),
#         unfocused_stem(q),
#         complex_k_type(q),
#         grammatical_cues_in_stem(q),
#         lost_sequence(q),
#         vague_terms(q),
#         more_than_one_correct(q),
#         negative_worded_stem(q)]
#     rows.append(r)
#     i += 1
#     print(i)

# columns = [
#     'ambiguous_unclear_information',
#     'implausible_distractors',
#     'none_of_the_above',
#     'longest_answer_correct',
#     'gratuitous_information_in_stem',
#     'true_or_false',
#     'avoid_convergence_cues',
#     'avoid_logical_cues',
#     'all_of_the_above',
#     'fill_in_the_blank',
#     'absolute_terms',
#     'word_repeats_in_stem_and_correct_answer',
#     'unfocused_stem',
#     'complex_k_type',
#     'grammatical_cues_in_stem',
#     'lost_sequence',
#     'vague_terms',
#     'more_than_one_correct',
#     'negative_worded_stem'
# ]

# oli_chemistry_results = pd.DataFrame(rows, columns=columns)
# oli_chemistry_results.to_csv("chem_results_auto_iwf.csv")

# #Print the number of "Falses", which correspond to IWF, found the table across each criteria
# for c in columns:
#     print(c, ': ', (~oli_chemistry_results[c]).values.sum())

opt_entity:  All of the above
lemma_nouns_opt:  []
ans_entity:  Proton
lemma_nouns_ans:  ['proton']
Question: Which particle weighs the least in an atom?
 Proton
Electron
Neutron
All of the above
Correct option: Proton
Quality: None
--- distractor not similar enough
Proton 		 All of the above 		 Score: 0.1279
Question: Which particle weighs the least in an atom?
 Proton
Electron
Neutron
All of the above
Correct option: Proton
Quality: None
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
cleaned_string:  Electron
cleaned_string:  Neutron
cleaned_string:  All of the above
--- GPT-3 believes the answer is incorrect:  Electron   Proton
1
opt_entity:  Rutherford
lemma_nouns_opt:  ['rutherford']
ans_entity:  Millikan
lemma_nouns_ans:  ['millikan']
*** low score, but they are the same entity ORG  &  ORG
Question: Which scientist discovered the fundamental electric charge, the charge of an electron?
 Millikan
Rutherford
Dalton
Thomson
Correct option: M

cleaned_string:  Robert  Millikan
cleaned_string:  George Stoney
cleaned_string:  Rutherford
15
Question: How can you determine the mass?
 the sum of protons and neutrons
sum of protons
sum of neurtrons
sum of electrons
Correct option: the sum of protons and neutrons
Quality: None
--- longest option is correct
repeating_nouns_synonyms:  ['proton']
--- we have a synonym of the answer being used in other answer choices, but not all of them:  proton
cleaned_string:  sum of protons
cleaned_string:  sum of neurtrons
cleaned_string:  sum of electrons
16
Question: which of these subatomic particles are neutral or uncharged in the atom?
 neutrons
electrons
protons
atom
Correct option: neutrons
Quality: None
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
repeating_nouns_synonyms:  []
cleaned_string:  electrons
cleaned_string:  protons
cleaned_string:  atom
17
Question: For Carbon to be neutral how many electrons must it have?
 2
3
4
6
Correct option: 2
Quality: None
repeating_nouns

cleaned_string:  AMU
cleaned_string:  Mass
cleaned_string:  atomic mass
31
Question: what is contained in the nucleus and what is outside of the nucleus
 neutrons only inside and protons outside
Protons and electrons inside and neutrons outside
protons  and neutrons  inside and electrons outside
electrons only inside and neutrons and protons outside
Correct option: neutrons only inside and protons outside
Quality: None
repeating_nouns_synonyms:  ['neutron', 'proton']
repeating_nouns_synonyms:  ['neutron', 'proton']
repeating_nouns_synonyms:  ['neutron', 'proton']
--- absolute word in question stem
cleaned_string:  Protons and electrons inside and neutrons outside
cleaned_string:  protons  and neutrons  inside and electrons outside
cleaned_string:  electrons only inside and neutrons and protons outside
--- GPT-3 believes the answer is incorrect:  protons  and neutrons  inside and electrons outside   neutrons only inside and protons outside
32
Question: What is the electrical charge of a

--- Question stem does not contain a question
cleaned_string:  Chemical
cleaned_string:  compounds
cleaned_string:  reactions
49
Question: What is the number of protons in the nucleus is called?
 the atomic number (Z)
the atomic number is (A)
the atomic number is (N)
the atomic number is (C)
Correct option: the atomic number (Z)
Quality: None
repeating_nouns_synonyms:  ['number']
repeating_nouns_synonyms:  ['number']
repeating_nouns_synonyms:  ['number']
cleaned_string:  the atomic number is 
cleaned_string:  the atomic number is N
cleaned_string:  the atomic number is 
50
ambiguous_unclear_information :  2
implausible_distractors :  6
none_of_the_above :  8
longest_answer_correct :  5
gratuitous_information_in_stem :  0
true_or_false :  3
avoid_convergence_cues :  12
avoid_logical_cues :  2
all_of_the_above :  0
fill_in_the_blank :  2
absolute_terms :  6
word_repeats_in_stem_and_correct_answer :  0
unfocused_stem :  2
complex_k_type :  0
grammatical_cues_in_stem :  3
lost_sequence :  

## OLI CollabU - 50 Questions

In [324]:
# qs = pd.read_csv('oli_collabu_questions.csv')
# qs = qs.fillna('')

# oli_collabu_student_questions = []
# for index, row in qs.iterrows():
#     question = MultipleChoiceQuestion(
#         stem=row['question'],
#         options=[row['altA'], row['altB'], row['altC'], row['altD']],
#         correct_option= row['altA']
#     )
#     oli_collabu_student_questions.append(question)

# rows = []
# i = 0
# for q in oli_collabu_student_questions:
#     r = [ambiguous_unclear_information(q),
#         implausible_distractors(q),
#         none_of_the_above(q),
#         longest_answer_correct(q),
#         gratuitous_information_in_stem(q),
#         true_or_false(q),
#         avoid_convergence_cues(q),
#         avoid_logical_cues(q),
#         all_of_the_above(q),
#         fill_in_the_blank(q),
#         absolute_terms(q),
#         word_repeats_in_stem_and_correct_answer(q),
#         unfocused_stem(q),
#         complex_k_type(q),
#         grammatical_cues_in_stem(q),
#         lost_sequence(q),
#         vague_terms(q),
#         more_than_one_correct(q),
#         negative_worded_stem(q)]
#     rows.append(r)
#     i += 1
#     print(i)

# columns = [
#     'ambiguous_unclear_information',
#     'implausible_distractors',
#     'none_of_the_above',
#     'longest_answer_correct',
#     'gratuitous_information_in_stem',
#     'true_or_false',
#     'avoid_convergence_cues',
#     'avoid_logical_cues',
#     'all_of_the_above',
#     'fill_in_the_blank',
#     'absolute_terms',
#     'word_repeats_in_stem_and_correct_answer',
#     'unfocused_stem',
#     'complex_k_type',
#     'grammatical_cues_in_stem',
#     'lost_sequence',
#     'vague_terms',
#     'more_than_one_correct',
#     'negative_worded_stem'
# ]

# oli_collabu_results = pd.DataFrame(rows, columns=columns)
# oli_collabu_results.to_csv("collabu_results_auto_iwf.csv")
# #Print the number of "Falses", which correspond to IWF, found the table across each criteria
# for c in columns:
#     print(c, ': ', (~oli_collabu_results[c]).values.sum())

## OLI Stats - 50 Questions

In [325]:
# qs = pd.read_csv('oli_stats_questions.csv')
# qs = qs.fillna('')

# oli_stats_student_questions = []
# for index, row in qs.iterrows():
#     question = MultipleChoiceQuestion(
#         stem=row['question'],
#         options=[row['altA'], row['altB'], row['altC'], row['altD']],
#         correct_option= row['altA']
#     )
#     oli_stats_student_questions.append(question)

# rows = []
# i = 0
# for q in oli_stats_student_questions:
#     r = [ambiguous_unclear_information(q),
#         implausible_distractors(q),
#         none_of_the_above(q),
#         longest_answer_correct(q),
#         gratuitous_information_in_stem(q),
#         true_or_false(q),
#         avoid_convergence_cues(q),
#         avoid_logical_cues(q),
#         all_of_the_above(q),
#         fill_in_the_blank(q),
#         absolute_terms(q),
#         word_repeats_in_stem_and_correct_answer(q),
#         unfocused_stem(q),
#         complex_k_type(q),
#         grammatical_cues_in_stem(q),
#         lost_sequence(q),
#         vague_terms(q),
#         more_than_one_correct(q),
#         negative_worded_stem(q)]
#     rows.append(r)
#     i += 1
#     print(i)

# columns = [
#     'ambiguous_unclear_information',
#     'implausible_distractors',
#     'none_of_the_above',
#     'longest_answer_correct',
#     'gratuitous_information_in_stem',
#     'true_or_false',
#     'avoid_convergence_cues',
#     'avoid_logical_cues',
#     'all_of_the_above',
#     'fill_in_the_blank',
#     'absolute_terms',
#     'word_repeats_in_stem_and_correct_answer',
#     'unfocused_stem',
#     'complex_k_type',
#     'grammatical_cues_in_stem',
#     'lost_sequence',
#     'vague_terms',
#     'more_than_one_correct',
#     'negative_worded_stem'
# ]

# oli_stats_results = pd.DataFrame(rows, columns=columns)
# oli_stats_results.to_csv("stats_results_auto_iwf.csv")
# #Print the number of "Falses", which correspond to IWF, found the table across each criteria
# for c in columns:
#     print(c, ': ', (~oli_stats_results[c]).values.sum())